In [ ]:
from pathlib import Path
from PIL import Image

# Process every 8th image and slice the top-left 256x256 cell
STEP = 8
SUB_W = 256
SUB_H = 256

base_dir = Path('/media/david/work_ssd_2_sata/python/R3GAN/runs/4_a100_6_days')
if not base_dir.exists():
    raise FileNotFoundError(f"Image folder not found: {base_dir}")


def slice_top_left_cell_256(image: Image.Image, sub_w: int = SUB_W, sub_h: int = SUB_H) -> Image.Image:
    # Ensure the crop doesn't exceed bounds
    crop_w = min(sub_w, image.width)
    crop_h = min(sub_h, image.height)
    return image.crop((0, 0, crop_w, crop_h))


# Collect images and process every Nth (STEP)
image_paths = sorted([p for p in base_dir.iterdir() if p.suffix.lower() in {'.png', '.jpg', '.jpeg'}])
selected_paths = image_paths[::STEP]

out_dir = base_dir / 'slices_r0c0'
out_dir.mkdir(exist_ok=True)

processed = 0

for img_path in selected_paths:
    with Image.open(img_path) as img:
        sub = slice_top_left_cell_256(img)
        out_path = out_dir / f"{img_path.stem}_r0c0{img_path.suffix}"
        sub.save(out_path)
        processed += 1

print(f"Processed {processed} images. Saved to: {out_dir}")



Processed 6 images. Saved to: /media/david/work_ssd_2_sata/python/R3GAN/runs/4_a100_6_days/slices_r0c0


In [8]:
from pathlib import Path
from PIL import Image

# Parameters
STEP = 2
CELL_W = 256
CELL_H = 256

base_dir = Path('/media/david/work_ssd_2_sata/python/R3GAN/runs/4_a100_6_days')
if not base_dir.exists():
    raise FileNotFoundError(f"Image folder not found: {base_dir}")


def slice_grid_cell(image: Image.Image, row: int = 0, col: int = 0, cell_w: int = CELL_W, cell_h: int = CELL_H) -> Image.Image:
    """Return the (row, col) 256x256 tile; defaults to top-left (0,0)."""
    left = col * cell_w
    upper = row * cell_h
    right = min(left + cell_w, image.width)
    lower = min(upper + cell_h, image.height)
    return image.crop((left, upper, right, lower))


def is_fake_image(path: Path) -> bool:
    """True for files named like fakes*.png."""
    name = path.name.lower()
    return name.startswith('fakes') and name.endswith('.png')


# Collect only fake images and process every Nth
image_paths = sorted(p for p in base_dir.iterdir() if p.is_file() and is_fake_image(p))
selected_paths = image_paths[::STEP]

out_dir = base_dir / 'slices_r0c0_fakes'
out_dir.mkdir(exist_ok=True)

processed = 0
for img_path in selected_paths:
    with Image.open(img_path) as img:
        sub = slice_grid_cell(img, row=0, col=0)
        out_path = out_dir / f"{img_path.stem}_r0c0{img_path.suffix}"
        sub.save(out_path)
        processed += 1

print(f"Processed {processed} fake images (every {STEP}th). Saved to: {out_dir}")



Processed 23 fake images (every 2th). Saved to: /media/david/work_ssd_2_sata/python/R3GAN/runs/4_a100_6_days/slices_r0c0_fakes
